# This notebook segments Toronto into various neighborhoods

# PART 1

In [35]:
import pandas as pd 
import numpy as np

URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
neighborhoods = pd.read_html(URL, header=0) #get info from wikipedia page
neighborhoods = neighborhoods[0] #remove unnecessary info

#remove rows where borough is not assigned
neighborhoods = neighborhoods.loc[(neighborhoods['Borough'] != 'Not assigned')] 

#confirm the postal codes are unique
print(neighborhoods.shape)
neighborhoods_count = neighborhoods.groupby('Postal Code')['Postal Code'].nunique()
print(neighborhoods_count.shape)

#ensure that neighborhoods are assigned as borough if not present
for id, row in neighborhoods.iterrows():
    if row.loc['Neighbourhood'] == 'Not assigned':
        neighborhoods.at[id, 'Neighbourhood'] = neighborhoods.at[id, 'Borough']
        print(str(row.loc['Borough']) + ' neighbourhood updated') 

#print the shape of the dataframe as requested in part 1
print('The shape of the neighborhoods dataframe is: ' + str(neighborhoods.shape))

(103, 3)
(103,)
The shape of the neighborhoods dataframe is: (103, 3)
